In [1]:
!pip install papermill

  Using cached click-8.1.7-py3-none-any.whl (97 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-win_amd64.whl (145 kB)
  Using cached entrypoints-0.4-py3-none-any.whl (5.3 kB)


You should consider upgrading via the 'C:\Users\sylx\project\birdis\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
import papermill as pm

pm.inspect_notebook('./windows_toasts.ipynb')


{'x': {'name': 'x',
  'inferred_type_name': 'float',
  'default': '1.0',
  'help': 'x is a float'},
 'y': {'name': 'y',
  'inferred_type_name': 'float',
  'default': '2.0',
  'help': 'y is a float'}}

In [7]:
pm.execute_notebook(
   './windows_toasts.ipynb',
   './windows_toasts_output.ipynb',
   parameters=dict(x=0.6, y=0.1)
)

c:\Users\sylx\project\birdis\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Executing: 100%|██████████| 3/3 [00:11<00:00,  3.87s/cell]


{'cells': [{'cell_type': 'code',
   'execution_count': 1,
   'metadata': {'tags': ['parameters'],
    'papermill': {'exception': False,
     'start_time': '2024-07-08T12:44:03.801638',
     'end_time': '2024-07-08T12:44:03.811652',
     'duration': 0.010014,
     'status': 'completed'},
    'execution': {'iopub.status.busy': '2024-07-08T12:44:03.804155Z',
     'iopub.execute_input': '2024-07-08T12:44:03.804155Z',
     'shell.execute_reply': '2024-07-08T12:44:03.810154Z',
     'iopub.status.idle': '2024-07-08T12:44:03.810653Z'}},
   'outputs': [],
   'source': 'x: float=1.0 # x is a float\ny: float=2.0 # y is a float',
   'id': '6d3f3f2e'},
  {'id': '0ddc7e96',
   'cell_type': 'code',
   'metadata': {'tags': ['injected-parameters'],
    'papermill': {'exception': False,
     'start_time': '2024-07-08T12:44:03.813664',
     'end_time': '2024-07-08T12:44:03.819668',
     'duration': 0.006004,
     'status': 'completed'},
    'execution': {'iopub.status.busy': '2024-07-08T12:44:03.815668Z'

# カスタムエンジンの実装
まず、Papermillのカスタムエンジンを作成します。このエンジンは、特定の変数を保持し、他のリソースを解放するように設計します。

In [12]:
import papermill as pm
from papermill.engines import NBClientEngine
from nbformat import NotebookNode

class CustomEngine(NBClientEngine):
    @classmethod
    def execute_managed_notebook(cls, nb_man, kernel_name, **kwargs):
        # 実行前に特定の変数を保持するためのセットアップ
        cls.preserved_variables = {}

        # Notebookの実行
        super().execute_managed_notebook(nb_man, kernel_name, **kwargs)

        # 実行後に特定の変数を保持し、他のリソースを解放
        for cell in nb_man.nb.cells:
            if cell.cell_type == 'code':
                # 特定の変数を保持
                exec(cell.source, globals())
                if 'preserve' in cell.metadata:
                    for var in cell.metadata['preserve']:
                        cls.preserved_variables[var] = globals().get(var)

        # 他のリソースを解放
        for var in list(globals().keys()):
            if var not in cls.preserved_variables:
                del globals()[var]

        # 保持した変数をグローバルに再設定
        globals().update(cls.preserved_variables)

# カスタムエンジンを登録
pm.engines.papermill_engines.register('custom_engine', CustomEngine)

#カスタムエンジンを使用してNotebookを実行
pm.execute_notebook(
    './windows_toasts.ipynb',
    './windows_toasts_output.ipynb',
    parameters=dict(x=0.6, y=0.1),
    engine_name='custom_engine'
)